In [ ]:
# RESTARTUJ KERNEL PO URUCHOMIENIU
!pip install --quiet instructor pydantic openai

In [ ]:
from pathlib import Path
import pandas as pd
import sqlite3
import base64
import json
from IPython.display import Image
from getpass import getpass
from datetime import date
import instructor
from pydantic import BaseModel
from openai import OpenAI

In [ ]:
# będziemy potrzebować klucza API do OpenAI żeby odczytać dane z faktur
openai_key = getpass("OpenAI key: ")

In [ ]:
openai_client = OpenAI(api_key=openai_key)

## Czytamy dane klientów

In [ ]:
# Czytamy plik sql
clients_df = pd.DataFrame()
with sqlite3.connect('clients.db') as conn:
     clients_df = pd.read_sql('SELECT * FROM clients', conn)

clients_df

## Czytamy dane produktów

In [ ]:
# Czytamy plik .csv
products_df = pd.DataFrame()
products_df = pd.read_csv('products.csv', sep=';')
products_df

In [ ]:
products2_df = products_df.drop(columns=['id'])
products2_df = products2_df.rename(columns={'name': 'product'})
products2_df['price'] = products2_df['price'].astype(float)

In [ ]:
products2_df

## Czytamy dane faktur

In [ ]:
DATA_PATH = Path('.')
PROCESSED_DATA_PATH = Path("Faktury") / "processed"

In [ ]:
#Sprawdzenie czy pliki .png z fakturami znajdują się w odpowiedniej ścieżce
for image_path in DATA_PATH.glob("*.png"):
    print(image_path)

In [ ]:
#Wyświetlenie jak wygląda przykładowa faktura
Image(DATA_PATH / "zad_domowe__invoice_1.png")

In [ ]:

class InvoiceInfoItem(BaseModel):
    description: str
    product_id: int
    quantity: int
    price: float


class InvoiceInfo(BaseModel):
    company_name: str
    customer_id: int
    customer_name: str
    invoice_number: int
    date: date
    # mamy tutaj do czynienia z listą elementów, więc danymi zagnieżdżonymi!
    items: list[InvoiceInfoItem]


instructor_openai_client = instructor.from_openai(OpenAI(api_key=openai_key))
orders = []
for image_path in sorted(DATA_PATH.glob("zad_domowe__invoice_*.png")):
    print(f"Processing {image_path}")
    with open(image_path, "rb") as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')

    
    invoice_info =  instructor_openai_client.chat.completions.create(
        model="gpt-4o-mini",
        #model="gpt-4o",
        response_model=InvoiceInfo,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Pobierz szczegóły o zakupionym produkcie, ilości, kliencie, cenie jednostkowej",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{image_data}",
                            "detail": "high"
                        },
                    },
                ],
            },
        ],
    )
    
    invoice_data = invoice_info.model_dump()
    for item in invoice_data["items"]:
        order = {
            "company_name": invoice_data["company_name"],
            "customer_id": invoice_data["customer_id"],
            "customer_name": invoice_data["customer_name"],
            "invoice_number": invoice_data["invoice_number"],
            "date": invoice_data["date"],
            "description": item["description"],
            "product_id": item["product_id"],
            "quantity": item["quantity"],
            "price": item["price"],
        }
        orders.append(order)

orders_df = pd.DataFrame(orders)

In [ ]:
orders_df

In [ ]:
orders_df_2= orders_df
orders_df_2 = orders_df_2.drop(columns=[
 'company_name',
 'customer_id',
 'invoice_number',
 'date',
 'product_id'])

In [ ]:
orders_df_2

In [ ]:
orders_df_2['description'].value_counts()

In [ ]:
orders_df_2.info()

## Łączenie danych

In [ ]:
orders_df_2['description'] = orders_df_2['description'].map({
    'router (id produktu 7)': 'router',
    'słuchawki (id produktu 5)': 'słuchawki',
    'ochraniacze (id produktu 16)': 'ochraniacze',
    'narty (id produktu 17)': 'narty',
    'rolki (id produktu 14)': 'rolki',
    'pendrive (id produktu 10)': 'pendrive',
    'laptop (id produktu 2)': 'laptop',
    'telewizor (id produktu 1)': 'telewizor',
    'klawiatura (id produktu 9)': 'klawiatura',
    'spodnie (id produktu 20)': 'spodnie',
    'spodnie': 'spodnie',
    'laptop': 'laptop',
    'klawiatura': 'klawiatura'
})
orders_df_2['description'].value_counts()

In [ ]:
orders_df_2 = orders_df_2.rename(columns={'customer_name': 'name','description': 'product'})
orders_df_2

In [ ]:
orders_products_df= pd.merge(orders_df_2, products2_df, on='product', how='outer')
orders_products_df

In [ ]:
orders_products_df = orders_products_df.drop(columns=['price_x'])

In [ ]:
orders_products_df = orders_products_df.rename(columns={'price_y': 'price'})

In [ ]:
orders_products_df

In [ ]:
orders_products_df = orders_products_df.dropna(subset = 'name')

In [ ]:
orders_products_df

In [ ]:
company_df = pd.DataFrame()
company_df = pd.merge(orders_df_2, clients_df, on='name', how='outer')

In [ ]:
company_df

In [ ]:
company_df = company_df.dropna(subset = 'product')
company_df

In [ ]:
company_df = company_df [['name', 'product', 'quantity', 'price']]
company_df

In [ ]:
company_df.to_csv('company.csv', index=False)